# Video files

## Analysis

The Mechwarrior 3 intro and campaign videos are found in the `video` folder on the CD. They can also optionally be installed to the hard drive, which is beneficial if the disc has DRM/weak sectors or simply for processing speed on modern solid-state drives. They are [AVI containers](https://en.wikipedia.org/wiki/Audio_Video_Interleave) (`*.avi`). [`ffmpeg`](https://www.ffmpeg.org/) is a great tool for analysing and encoding videos, although it can be tricky to use. First, analysis of the files using `ffprobe`:

```bash
for f in *.avi; do ffprobe -hide_banner "$f"; done
```

The file checksums between the US versions 1.0, 1.1, and 1.2 are exactly the same (on the CD - I don't think the patch affects the video files, simply based on the size, but haven't checked).

The video streams are encoded using Intel's [Indeo codec](https://en.wikipedia.org/wiki/Indeo) (version 5, FourCC `IV50`). They are all 640x480 at 15 frames per second, although the bitrates vary from 3020 kb/s to 1260 kb/s. The audio streams are raw [pulse-code modulation (PCM)](https://en.wikipedia.org/wiki/Pulse-code_modulation) at 22050 Hz, so uncompressed.

For the US version, most audio streams are mono with unsigned 8-bit samples, resulting in a bitrate of 176 kb/s. Two files (`c1.avi` and `intro.avi`) are stereo with little-endian signed 16-bit samples, resulting in a much higher bitrate of 705 kb/s. For the German version, the opposite is true. Most are stereo with little-endian signed 16-bit samples, while `intro.avi` and `c4win.avi` are mono unsigned 8-bit. Interestingly, these have the metadata "Sound Forge 4.0 Audio" attached, which was a [German sound editing program](https://en.wikipedia.org/wiki/Sound_Forge), probably used by the localisation team.

These codecs were no doubt chosen because they could be decoded with very little CPU, not because of their quality. This is especially true if they had to be streamed from the CD. Codecs have come far since then, with ubiquitous hardware support. I think re-encoding these files is the best option for preservation. Indeo has at least one [vulnerability](https://support.microsoft.com/en-us/help/954157/microsoft-security-advisory-vulnerabilities-in-the-indeo-codec-could-a), meaning the codec is unlikely to be installed on modern systems.

## Re-encoding

To compress the audio, there are several options. If supported, [advanced audio coding (ACC)](https://en.wikipedia.org/wiki/Advanced_Audio_Coding) is excellent at low bitrates, and for mainly speech, using 64 kb/s is fine without any concerns of quality loss. The command line options are `-codec:a aac -b:a 64k` *. AAC is patented and not all game engines support it. This is generally problematic for good audio codecs. A viable alternative is to not alter the audio and just copy it using `-codec:a copy`, as raw PCM support is ubiquitous.

As mentioned, I definitely want to re-encode the video because of known Indeo vulnerabilities. H.264/x264 is widely supported. Quality-wise, it's a bit trickier than the audio, because it's more subjective in comparisons. The original video is highly compressed, with visible compression artefacts - please keep this in mind, the re-encoded file can't be better than the original. So personally, I find the video re-encoded with a low bitrate fine. In fact, choosing a low bitrate smooths some of the original, block-y compression artefacts out (the smoothing could be done via processing at higher bitrates). But you can decide for yourself, in a minute I'll show how to compare the re-encoded to the original. And worst case, files can be re-encoded from the original again.

My recommendation is to use a fairly quick encoding to test things out, and a low quality factor. Something like `-codec:v libx264 -preset medium -crf 28`. It's worth reading the [`ffmpeg` H.264 encoding guide](https://trac.ffmpeg.org/wiki/Encode/H.264) if you wish to change these parameters. Choose a slower preset should deliver the same quality at a lower bitrate, at expense of encoding time. Choosing a lower `crf` value will increase the bitrate, which in theory increases quality. Given the source material, that probably won't do much those. Once you're happy with the parameters, I'd suggest using a slower preset for the final encoding, like `veryslow`, since processing power is cheap and these videos are short and have a tiny resolution (generally, the preset doesn't affect quality very much).

For a container format with maximum compatibility, I've chosen [MPEG-4](https://en.wikipedia.org/wiki/MPEG-4_Part_14) (`*.mp4`), although if supported by your use-case, the open standard [Matroska](https://matroska.org/) (`*.mkv`) is an excellent choice.

---

\* `libfdk` [might be slightly higher in quality](https://trac.ffmpeg.org/wiki/Encode/AAC), and if your build of `ffmpeg` was compiled with `libfdk` support you could try using the `libfdk_aac` codec. That also enabled the use of variable bit rate. However, I don't think it's worth the effort. The input isn't exactly high quality in the first place, and the built-in AAC encoder is pretty good.

### TL;DR

```bash
for f in *.avi
do
    ffmpeg -i "$f" -codec:v "libx264" -preset "medium" -crf "30" -codec:a "aac" -b:a "64k" "${f%.*}.mp4"
done
```

## Comparing results

The [MPV](https://mpv.io/) media player [can play two (or more) videos side-by-side](https://superuser.com/a/1325668), which is great for comparing the encoded video. 

```bash
mpv --lavfi-complex="[vid1][vid2]hstack[vo]" intro.avi --external-file=intro.mp4
```

## Futher analysis

It's interesting to see just how far codecs have come. For those settings, the average reduction in size is 86.5% for the US version and almost 89% for the German version!

In [1]:
from pathlib import Path


def savings(path):
    video_path = Path(path)
    print("---", video_path, "---")

    def mb(size):
        return size / 1024 / 1024

    avg = 0.0
    count = 0
    for avi in video_path.glob("*.avi"):
        mp4 = avi.with_suffix(".mp4").resolve(strict=True)
        avi_size = avi.stat().st_size
        mp4_size = mp4.stat().st_size
        percentage = mp4_size / avi_size
        avg += percentage
        count += 1
        print(
            f"{avi.name:<12} {mb(avi_size):>5.2f} MiB -> {mb(mp4_size):>5.2f} MiB ({1 - percentage:>5.1%})"
        )
    avg = avg / count
    print(f"avg: {1 - avg:>5.1%}")


savings("video/v1.0-de")
savings("video/v1.0-us")

--- video/v1.0-de ---
c1.avi       13.45 MiB ->  1.36 MiB (89.9%)
c1m1.avi     10.88 MiB ->  0.97 MiB (91.1%)
c1m2.avi      7.44 MiB ->  0.77 MiB (89.6%)
c1m3.avi      6.50 MiB ->  0.74 MiB (88.5%)
c1m4.avi     11.38 MiB ->  1.16 MiB (89.8%)
c2.avi       13.32 MiB ->  1.67 MiB (87.5%)
c2m1.avi      5.95 MiB ->  0.65 MiB (89.1%)
c2m2.avi     12.86 MiB ->  1.22 MiB (90.5%)
c2m3.avi      7.86 MiB ->  0.75 MiB (90.5%)
c2m4.avi      9.46 MiB ->  0.79 MiB (91.6%)
c3.avi        6.88 MiB ->  1.62 MiB (76.5%)
c3m1.avi      7.38 MiB ->  1.06 MiB (85.6%)
c3m2.avi      7.75 MiB ->  1.02 MiB (86.8%)
c3m4.avi      9.27 MiB ->  1.13 MiB (87.9%)
c3m5.avi     11.80 MiB ->  1.08 MiB (90.9%)
c3m6.avi      7.15 MiB ->  0.84 MiB (88.3%)
c4win.avi    23.98 MiB ->  1.50 MiB (93.8%)
Campaign.avi 77.76 MiB -> 11.35 MiB (85.4%)
intro.avi    76.00 MiB ->  5.33 MiB (93.0%)
avg: 88.7%
--- video/v1.0-us ---
c1.avi       14.50 MiB ->  1.36 MiB (90.6%)
c1m1.avi      8.75 MiB ->  0.97 MiB (88.9%)
c1m2.avi      5.96 Mi

## Next up

[Installation and version comparison](04-installation.ipynb)